In [3]:
import pandas
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras import optimizers
from keras import losses
from sklearn import preprocessing

corrected = "./data/corrected"
kddcup_data_corrected = "./data/kddcup.data.corrected"

kddcup_data_10_percent_corrected = "./data/kddcup.data_10_percent_corrected"
kddcup_newtestdata_10_percent_unlabeled = "./data/kddcup.newtestdata_10_percent_unlabeled"

kddcup_testdata_unlabeled = "./data/kddcup.testdata.unlabeled"
kddcup_testdata_unlabeled_10_percent = "./data/kddcup.testdata.unlabeled_10_percent"

correspondence = "./data/training_attack_types.txt"

Using TensorFlow backend.


In [4]:
dct = dict()
with open(correspondence, "r") as f:
    for line in f:
        line = line.split(' ')
        dct[line[0]] = line[1][:-1]
dct["normal"] = "normal"

In [5]:
data = pandas.read_csv(kddcup_data_10_percent_corrected, header=None)
data[41].value_counts()
statistics = {"dos": 0, "u2r": 0, "probe": 0, "r2l": 0, "normal": 0}
labelling  = {"dos": 1, "u2r": 2, "probe": 3, "r2l": 4, "normal": 0}

for i in range(data.shape[0]):
    intrusion_class = dct[data.iat[i, 41][:-1]]
    data.iat[i, 41] = labelling[intrusion_class]
    statistics[intrusion_class] +=1
    
statistics

{'dos': 391458, 'normal': 97278, 'probe': 4107, 'r2l': 1126, 'u2r': 52}

In [6]:
le = preprocessing.LabelEncoder()
need_labeling = [1, 2, 3]

for i in need_labeling:
    le.fit(data[i])
    data[i] = le.transform(data[i])

In [7]:
n_dos = 500
n_r2l = 500
n_probe = 500
n_normal = 1500
n_u2r = 50

data_for_training = pandas.DataFrame()
data_for_training = data_for_training.append(data[data[41] == labelling['dos']].sample(n_dos), ignore_index=True)
data_for_training = data_for_training.append(data[data[41] == labelling['r2l']].sample(n_r2l), ignore_index=True)
data_for_training = data_for_training.append(data[data[41] == labelling['probe']].sample(n_probe), ignore_index=True)
data_for_training = data_for_training.append(data[data[41] == labelling['u2r']].sample(n_u2r), ignore_index=True)
data_for_training = data_for_training.append(data[data[41] == labelling['normal']].sample(n_normal), ignore_index=True)
data_for_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3050 entries, 0 to 3049
Data columns (total 42 columns):
0     3050 non-null int64
1     3050 non-null int64
2     3050 non-null int64
3     3050 non-null int64
4     3050 non-null int64
5     3050 non-null int64
6     3050 non-null int64
7     3050 non-null int64
8     3050 non-null int64
9     3050 non-null int64
10    3050 non-null int64
11    3050 non-null int64
12    3050 non-null int64
13    3050 non-null int64
14    3050 non-null int64
15    3050 non-null int64
16    3050 non-null int64
17    3050 non-null int64
18    3050 non-null int64
19    3050 non-null int64
20    3050 non-null int64
21    3050 non-null int64
22    3050 non-null int64
23    3050 non-null int64
24    3050 non-null float64
25    3050 non-null float64
26    3050 non-null float64
27    3050 non-null float64
28    3050 non-null float64
29    3050 non-null float64
30    3050 non-null float64
31    3050 non-null int64
32    3050 non-null int64
33    3050 non-null f

In [8]:
test_data = pandas.read_csv(kddcup_data_10_percent_corrected, header=None)

for i in range(test_data.shape[0]):
    intrusion_class = dct[test_data.iat[i, 41][:-1]]
    test_data.iat[i, 41] = labelling[intrusion_class]
    
need_labeling = [1, 2, 3]

for i in need_labeling:
    le.fit(test_data[i])
    test_data[i] = le.transform(test_data[i])
    
test_data[41].value_counts()

1    391458
0     97278
3      4107
4      1126
2        52
Name: 41, dtype: int64

In [9]:
min_max_scaler = preprocessing.MinMaxScaler()

x_train = data_for_training.as_matrix(range(41))
x_train = min_max_scaler.fit_transform(x_train)

y_train = data_for_training.as_matrix([41])

In [10]:
x_test = test_data.as_matrix(range(41))
x_test = min_max_scaler.fit_transform(x_test)

y_test = test_data.as_matrix([41])

In [11]:
hidden_layer_size = 60
timestep = 50
batch_size = 50
epochs = 15
learning_rate = 0.01
x_train = np.reshape(x_train, (-1, 50, 41))
y_train = np.reshape(y_train, (3050, 1, 1))

In [12]:
#model = Sequential()
#model.add(LSTM(hidden_layer_size, input_shape=x_train.shape[1:], return_sequences=True))
#model.add(LSTM(hidden_layer_size, return_sequences=True))
#model.add(Dense(1, activation='softmax'))

#sgd = optimizers.SGD(lr=learning_rate)
#model.compile(loss=losses.mean_squared_error,
#              optimizer=sgd,
#              metrics=['accuracy'])
#
#model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)
#score = model.evaluate(x_test, y_test, batch_size=)

Epoch 1/15
3050/3050 [==============================] - 10s - loss: 2.6393 - acc: 0.1639    
Epoch 2/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 3/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 4/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 5/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 6/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 7/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 8/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 9/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 10/15
3050/3050 [==============================] - 0s - loss: 2.6393 - acc: 0.1639     
Epoch 11/15
3050/3050 [==============================] - 0s - loss: 2